# NEPOOL IDR Drop

Email scrape logins, webscrape data, filter for IDR drop.

In [1]:
import os
basepath = 'C:\\Users\wb5888\la-tools-test\IDR_Drop'
os.chdir(basepath)
print('working in directory {}.'.format(basepath))

from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
import selenium.webdriver.support.ui as ui
from selenium.webdriver.common.keys import Keys
import selenium.webdriver as webdriver
from pandas.io.json import json_normalize
import datetime as dt
import pandas as pd
import numpy as np
import IDRdrop
import EPOwebscrape
import emailscrape
import json

working in directory C:\Users\wb5888\la-tools-test\IDR_Drop.
226 aaron emails, 1002 epo emails 403 ngrid emails
parsing aaron.downing@eversource.com inbox

scraping emails...
parsing epoadmin@eversource.com

scraping emails...
parsing michael.stanton@nationalgrid.com

scraping emails...
error with 0 of 3 inboxes
found  1467  new emails: 
writing .json object
file saved as email_bodies_10_11_2019.json


## Save Login Info

In [2]:
filename = 'email_bodies_10_11_2019.json'

In [3]:
with open(filename, 'r') as f:
    bodies = json.load(f)

    bodies = json.loads(bodies)

good, fname = emailscrape.bodies_json(bodies)

print('converted .json to dataframe')

converted .json to dataframe


## Read to .csv

In [6]:
good = emailscrape.past_days(good, 5)
good.reset_index(drop = True, inplace = True)
good

,accts,date,name,pw,user,util
0,"[51420574014_465861007, 51452223035_040812006,...",2019-10-11 11:36:04,Hartford Healthcare Corporation,78cck,harth-engie,CLP
1,[54202385001_052931000],2019-10-11 11:34:00,Protective Armored Systems,m372b,usar-engie,WMECO
2,"[51420574014_465861007, 51452223035_040812006,...",2019-10-11 10:34:43,Hartford Healthcare Corporation,78cck,harth-engie,CLP
3,"[54977381078_126733008, 54480402023_526541004,...",2019-10-10 15:55:06,Berkshire Medical Center Inc.,d823k,berkmc-engie,WMECO
4,[51436492052_684002003],2019-10-10 15:54:30,Teds Food Center,tc836,teds-engie,CLP
5,"[51549893006_482562007, 51874553092_860042003]",2019-10-10 09:43:48,PM Engineered Solutions,p899g,pme-engie,CLP
6,[51482323003_190712004],2019-10-09 13:12:03,Bimba Bakeries,9567c,bbak-engie,CLP
7,"[0077868038, 2716743003]",2019-10-11 15:22:23,"[STOP AND SHOP, STOP SHOP]",3824,SUEZ_HIST,NGRID
8,"[1448303009, 7789737094]",2019-10-11 14:53:59,"[FAIRVIEW HOSPITAL, BERKSHIRE MEDICAL CE]",3824,SUEZ_HIST,NGRID


In [5]:
outfile = filename.split('.')[0]
outfile = '.'.join([outfile, 'xlsx'])
good.to_excel(outfile, index = False)
print('wrote {} to {}'.format(outfile, basepath))

wrote email_bodies_10_11_2019.xlsx to C:\Users\wb5888\la-tools-test\IDR_Drop


## Choose account to webscrape

In [8]:
accts = [7,8]
dl_files = good.iloc[accts,:]
dl_files.reset_index(inplace = True, drop = True)
dl_files.head()

,accts,date,name,pw,user,util
0,"[0077868038, 2716743003]",2019-10-11 15:22:23,"[STOP AND SHOP, STOP SHOP]",3824,SUEZ_HIST,NGRID
1,"[1448303009, 7789737094]",2019-10-11 14:53:59,"[FAIRVIEW HOSPITAL, BERKSHIRE MEDICAL CE]",3824,SUEZ_HIST,NGRID


## Download files from EPO portal

In [ ]:
result = []

for row in range(0, len(dl_files)):
    try:
        EPOwebscrape.idr_download(row, dl_files)
        result.append(True)
        print('done with {} of {}.'.format(row+1, len(dl_files)))
        print('----------------------------')
        
    except:
        print('error, accts {}.'.format(dl_files.accts[row]))
        result.append(False)
        print('done with {} of {}.'.format(row+1, len(dl_files.accts)))
        print('----------------------------')

print('writing results to csv...')
good['success'] = result
good.to_csv('portal_dl_results_101119.csv')
print('success with {} of {} accounts.'.format(np.sum(good.success), len(good.success)))

user:  SUEZ_HIST
password:  3824
logging on...
set to last  400  days.
logging on...
looking for accts ['0077868038', '2716743003'].


C:\Users\wb5888\la-tools-test\IDR_Drop\EPOwebscrape.py:167: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 167 of the file C:\Users\wb5888\la-tools-test\IDR_Drop\EPOwebscrape.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(browser.page_source)


found items in portal.
trying search & download...


## Show downloaded files from EPO portal

Here *filepath* is a directory containing downloaded EPO files. Code will print 20 most recent files.

In [9]:
#filepath = os.getcwd()
readpath = 'C:\\Users\\wb5888\\la-tools-test\\IDR_Drop\\Downloads'

myfiles = IDRdrop.show_dir(readpath, 20)
print(myfiles)

files found in dir:  C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads
                                                files                time
0   eversource_febb7abf-0041-4c06-91f5-9cf4179c607... 2019-10-11 12:56:54
1   eversource_febb7abf-0041-4c06-91f5-9cf4179c607... 2019-10-11 12:56:38
2   eversource_febb7abf-0041-4c06-91f5-9cf4179c607... 2019-10-11 12:56:23
3   eversource_f7c3246b-ab2e-48d5-8ac4-fa1dbb998fb... 2019-10-11 12:55:28
4   eversource_f7c3246b-ab2e-48d5-8ac4-fa1dbb998fb... 2019-10-11 12:55:11
5   eversource_f7c3246b-ab2e-48d5-8ac4-fa1dbb998fb... 2019-10-11 12:54:55
6   ngrid_734a00df-5486-4d6c-b095-435f56856a49_2eb... 2019-10-07 14:39:07
7   eversource_2bd29087-a360-4275-9d60-951b0ee0977... 2019-10-07 14:36:22
8   eversource_8fcc6582-4202-4cfd-821d-ed4430171bf... 2019-09-30 11:22:51
9   eversource_48ceb8c3-51bf-4c54-8331-9150bd03501... 2019-09-30 11:21:56
10  eversource_86c72163-24b5-472c-a4e9-266fb107ae2... 2019-09-30 11:21:37


Choose files to split into Raw IDR files.

In [20]:
index = [0]

splitfiles = list(myfiles.files[:6])
print('files to split: ')
splitfiles

files to split: 


['eversource_febb7abf-0041-4c06-91f5-9cf4179c6074_0fb2a7d3_hourlycsv.csv',
 'eversource_febb7abf-0041-4c06-91f5-9cf4179c6074_5c97a4e4_hourlycsv.csv',
 'eversource_febb7abf-0041-4c06-91f5-9cf4179c6074_01933c96_hourlycsv.csv',
 'eversource_f7c3246b-ab2e-48d5-8ac4-fa1dbb998fbd_df4237ee_hourlycsv.csv',
 'eversource_f7c3246b-ab2e-48d5-8ac4-fa1dbb998fbd_bcfd71a5_hourlycsv.csv',
 'eversource_f7c3246b-ab2e-48d5-8ac4-fa1dbb998fbd_5e41f9b3_hourlycsv.csv']

## Batch process downloaded EPO files into Raw IDRs

In [24]:
readpath = 'C:\\Users\\wb5888\\la-tools-test\\IDR_Drop\\Downloads'
writepath = 'C:\\Users\\wb5888\\la-tools-test\\IDR_Drop\\RAW_IDR'

utility = 'CLP'
accts = dl_files['accts']

for file in splitfiles:
    os.chdir(readpath)
        
    for acct in accts:
        for b in acct:
            fname = ''.join([utility, '_', b, '_IDR_RAW.csv'])
            filedf = pd.read_csv(file, sep = ",", header = 0)
    
            IDRdrop.raw_split(filedf, readpath, writepath, utility, os.path.join(readpath, fname))
        print('success, file: ', file)
        
        

error_log = []
        
print('RAW split complete.')

found 4 accounts.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51420574014_465861007_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51420574014_465861007_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51420574014_465861007_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51420574014_465861007_IDR_RAW.csv  written.
success, file:  eversource_febb7abf-0041-4c06-91f5-9cf4179c6074_0fb2a7d3_hourlycsv.csv
found 4 accounts.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51452223035_040812006_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51452223035_040812006_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51452223035_040812006_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51452223035_040812006_IDR_RAW.csv  written.
success, file:  eversource_febb7abf-0041-4c06-91f5-9cf4179c6074_0fb2a7d3_hourlycsv.csv
found 4 accoun

C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51505275024_323761006_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51505275024_323761006_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51505275024_323761006_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51505275024_323761006_IDR_RAW.csv  written.
success, file:  eversource_febb7abf-0041-4c06-91f5-9cf4179c6074_0fb2a7d3_hourlycsv.csv
found 4 accounts.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51511694010_317103008_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51511694010_317103008_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51511694010_317103008_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51511694010_317103008_IDR_RAW.csv  written.
success, file:  eversource_febb7abf-0041-4c06-91f5-9cf4179c6074_0fb2a7d3_hourlycsv.csv
found 4 accounts.
C:\Users\wb588

C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51640724076_352382007_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51640724076_352382007_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51640724076_352382007_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51640724076_352382007_IDR_RAW.csv  written.
success, file:  eversource_febb7abf-0041-4c06-91f5-9cf4179c6074_5c97a4e4_hourlycsv.csv
found 4 accounts.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51733934061_347092002_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51733934061_347092002_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51733934061_347092002_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51733934061_347092002_IDR_RAW.csv  written.
success, file:  eversource_febb7abf-0041-4c06-91f5-9cf4179c6074_5c97a4e4_hourlycsv.csv
found 4 accounts.
C:\Users\wb588

C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51997893060_648243008_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51997893060_648243008_IDR_RAW.csv  written.
success, file:  eversource_febb7abf-0041-4c06-91f5-9cf4179c6074_5c97a4e4_hourlycsv.csv
found 4 accounts.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51420574014_465861007_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51420574014_465861007_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51420574014_465861007_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51420574014_465861007_IDR_RAW.csv  written.
success, file:  eversource_febb7abf-0041-4c06-91f5-9cf4179c6074_01933c96_hourlycsv.csv
found 4 accounts.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51452223035_040812006_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51452223035_040812006_IDR_RAW.csv  written.
C:\Users\wb588

C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51492834064_460192005_IDR_RAW.csv  written.
success, file:  eversource_febb7abf-0041-4c06-91f5-9cf4179c6074_01933c96_hourlycsv.csv
found 4 accounts.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51505275024_323761006_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51505275024_323761006_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51505275024_323761006_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51505275024_323761006_IDR_RAW.csv  written.
success, file:  eversource_febb7abf-0041-4c06-91f5-9cf4179c6074_01933c96_hourlycsv.csv
found 4 accounts.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51511694010_317103008_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51511694010_317103008_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51511694010_317103008_IDR_RAW.csv  written.
C:\Users\wb588

C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51640724076_352382007_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51640724076_352382007_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51640724076_352382007_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51640724076_352382007_IDR_RAW.csv  written.
success, file:  eversource_f7c3246b-ab2e-48d5-8ac4-fa1dbb998fbd_df4237ee_hourlycsv.csv
found 4 accounts.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51733934061_347092002_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51733934061_347092002_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51733934061_347092002_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51733934061_347092002_IDR_RAW.csv  written.
success, file:  eversource_f7c3246b-ab2e-48d5-8ac4-fa1dbb998fbd_df4237ee_hourlycsv.csv
found 4 accounts.
C:\Users\wb588

C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51472893064_344143007_IDR_RAW.csv  written.
success, file:  eversource_f7c3246b-ab2e-48d5-8ac4-fa1dbb998fbd_bcfd71a5_hourlycsv.csv
found 4 accounts.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51492834064_460192005_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51492834064_460192005_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51492834064_460192005_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51492834064_460192005_IDR_RAW.csv  written.
success, file:  eversource_f7c3246b-ab2e-48d5-8ac4-fa1dbb998fbd_bcfd71a5_hourlycsv.csv
found 4 accounts.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51505275024_323761006_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51505275024_323761006_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51505275024_323761006_IDR_RAW.csv  written.
C:\Users\wb588

C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51546023086_133612007_IDR_RAW.csv  written.
success, file:  eversource_f7c3246b-ab2e-48d5-8ac4-fa1dbb998fbd_5e41f9b3_hourlycsv.csv
found 4 accounts.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51601665094_750042003_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51601665094_750042003_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51601665094_750042003_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51601665094_750042003_IDR_RAW.csv  written.
success, file:  eversource_f7c3246b-ab2e-48d5-8ac4-fa1dbb998fbd_5e41f9b3_hourlycsv.csv
found 4 accounts.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51640724076_352382007_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51640724076_352382007_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51640724076_352382007_IDR_RAW.csv  written.
C:\Users\wb588

C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51801694019_298103007_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51801694019_298103007_IDR_RAW.csv  written.
success, file:  eversource_f7c3246b-ab2e-48d5-8ac4-fa1dbb998fbd_5e41f9b3_hourlycsv.csv
found 4 accounts.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51970204004_216762005_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51970204004_216762005_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51970204004_216762005_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51970204004_216762005_IDR_RAW.csv  written.
success, file:  eversource_f7c3246b-ab2e-48d5-8ac4-fa1dbb998fbd_5e41f9b3_hourlycsv.csv
found 4 accounts.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51997893060_648243008_IDR_RAW.csv  written.
C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads\CLP_51997893060_648243008_IDR_RAW.csv  written.
C:\Users\wb588

## Show Raw IDR files based on utility

Here *rawpath* is directory containing Raw IDRs - 25 most recent will be shown.

In [25]:
rawfiles = IDRdrop.show_dir(writepath, 30)
print(rawfiles)

files found in dir:  C:\Users\wb5888\la-tools-test\IDR_Drop\Downloads
                                                files                time
0               CLP_51997893060_648243008_IDR_RAW.csv 2019-10-11 13:22:51
1               CLP_51801694019_298103007_IDR_RAW.csv 2019-10-11 13:22:50
2               CLP_51970204004_216762005_IDR_RAW.csv 2019-10-11 13:22:50
3               CLP_51601665094_750042003_IDR_RAW.csv 2019-10-11 13:22:49
4               CLP_51640724076_352382007_IDR_RAW.csv 2019-10-11 13:22:49
5               CLP_51733934061_347092002_IDR_RAW.csv 2019-10-11 13:22:49
6               CLP_51505275024_323761006_IDR_RAW.csv 2019-10-11 13:22:48
7               CLP_51511694010_317103008_IDR_RAW.csv 2019-10-11 13:22:48
8               CLP_51546023086_133612007_IDR_RAW.csv 2019-10-11 13:22:48
9               CLP_51452223035_040812006_IDR_RAW.csv 2019-10-11 13:22:47
10              CLP_51472893064_344143007_IDR_RAW.csv 2019-10-11 13:22:47
11              CLP_51492834064_460192005_

Choose Raw IDRs to filter into IDR files.

In [29]:
index = [36]

processfiles = list(rawfiles.files[:13])

print('files to be processed: ')
processfiles

files to be processed: 


['CLP_51997893060_648243008_IDR_RAW.csv',
 'CLP_51801694019_298103007_IDR_RAW.csv',
 'CLP_51970204004_216762005_IDR_RAW.csv',
 'CLP_51601665094_750042003_IDR_RAW.csv',
 'CLP_51640724076_352382007_IDR_RAW.csv',
 'CLP_51733934061_347092002_IDR_RAW.csv',
 'CLP_51505275024_323761006_IDR_RAW.csv',
 'CLP_51511694010_317103008_IDR_RAW.csv',
 'CLP_51546023086_133612007_IDR_RAW.csv',
 'CLP_51452223035_040812006_IDR_RAW.csv',
 'CLP_51472893064_344143007_IDR_RAW.csv',
 'CLP_51492834064_460192005_IDR_RAW.csv',
 'CLP_51420574014_465861007_IDR_RAW.csv']

## Batch filter Raw IDR into IDR files to be dropped

In [10]:
error_log

['26281241005_MECO_IDR_RAW.csv',
 '26281131008_MECO_IDR_RAW.csv',
 '26281221007_MECO_IDR_RAW.csv',
 '26281231006_MECO_IDR_RAW.csv',
 '26280821005_MECO_IDR_RAW.csv',
 '28307660010_MECO_IDR_RAW.csv',
 '29001880011_MECO_IDR_RAW.csv',
 '25835511020_MECO_IDR_RAW.csv',
 '25745431046_MECO_IDR_RAW.csv',
 '288552094_MECO_IDR_RAW.csv',
 '25651191022_MECO_IDR_RAW.csv']

In [43]:
readpath = 'C:\\Users\\wb5888\\la-tools-test\\IDR_Drop\\RAW_IDR\\'
writepath = 'C:\\Users\\wb5888\\la-tools-test\\IDR_Drop\\IDR\\'
error_log = []

for dropfile in processfiles:
    try:
        IDRdrop.data_drop(os.path.join(readpath, dropfile), readpath, writepath)
        print('success, file: ', dropfile)
    
    except:
        error_log.append(dropfile)
        print("error, file: ", dropfile)

spot check output file at date: 7/23/2019.
error, file:  CLP_51997893060_648243008_IDR_RAW.csv
spot check output file at date: 7/23/2019.
error, file:  CLP_51801694019_298103007_IDR_RAW.csv
spot check output file at date: 7/23/2019.
error, file:  CLP_51970204004_216762005_IDR_RAW.csv
spot check output file at date: 7/23/2019.
error, file:  CLP_51601665094_750042003_IDR_RAW.csv
spot check output file at date: 7/23/2019.
error, file:  CLP_51640724076_352382007_IDR_RAW.csv
spot check output file at date: 7/23/2019.
error, file:  CLP_51733934061_347092002_IDR_RAW.csv
spot check output file at date: 7/23/2019.
error, file:  CLP_51505275024_323761006_IDR_RAW.csv
spot check output file at date: 7/23/2019.
error, file:  CLP_51511694010_317103008_IDR_RAW.csv
spot check output file at date: 7/23/2019.
error, file:  CLP_51546023086_133612007_IDR_RAW.csv
spot check output file at date: 7/23/2019.
error, file:  CLP_51452223035_040812006_IDR_RAW.csv
spot check output file at date: 7/23/2019.
error, 